# Evaluación de Modelos con Conjunto de Datos Propio

## Paso 1: Cargar las Librerías

In [18]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from datasets import load_dataset
import evaluate
from tqdm import tqdm
import json
import requests

In [ ]:
# URLs of the files
urls = {
    "eval_colombia_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/eval_colombia_mexico_dataset.json",
    "train_colombia_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/train_colombia_mexico_dataset.json",
    "train_colombia_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/train_colombia_dataset.json",
    "train_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/train_mexico_dataset.json",
    "eval_colombia_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/eval_colombia_dataset.json",
    "eval_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/eval_mexico_dataset.json",
}

# Dictionary to store the loaded JSON data
datasets = {}

for filename, url in urls.items():
    # Download the file
    response = requests.get(url)
    if response.status_code == 200:
        # Save locally
        with open(filename, "wb") as f:
            f.write(response.content)
        # Load JSON into Python
        datasets[filename] = response.json()
        print(f"{filename} downloaded and loaded successfully!")
    else:
        print(f"Failed to download {filename}. Status code: {response.status_code}")

## Paso 2: Cargar los modelos

In [19]:
models = {
    "BERT (Base)": "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",
    "DistilBERT (Base)": "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",
    "BERT (Fine-tuned News QA)": "BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico",
    "DistilBERT (Fine-tuned News QA)": "BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico",
}

## Paso 3: Cargar los conjuntos de datos

In [20]:
GOLD_PATH   = "" #"../datasets/exploration_datasets/gold/"

In [21]:
datasets = {
    "Colombia and Mexico Train": GOLD_PATH + "train_colombia_mexico_dataset.json",
    "Colombia and Mexico Eval": GOLD_PATH + "eval_colombia_mexico_dataset.json",
    "Colombia Train": GOLD_PATH + "train_colombia_dataset.json",
    "Mexico Train": GOLD_PATH + "train_mexico_dataset.json",
    "Colombia Eval": GOLD_PATH + "eval_colombia_dataset.json",
    "Mexico Eval": GOLD_PATH + "eval_mexico_dataset.json"
}

## Paso 4: Cargar las métricas

In [22]:
# 🧮 Metric (SQuAD v2)
metric = evaluate.load("squad_v2")

## Paso 5: Transformar el conjunto de datos a español: Aplanar a formato Contexto + Preguntas y Respuestas (QA

In [23]:
def flatten_squad(dataset):
    # Handle "data" key if present
    if isinstance(dataset, dict) and "data" in dataset:
        dataset = dataset["data"]

    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "title": article.get("title", ""),
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data

## Paso 5: Evaluar el Modelo

In [24]:
def evaluate_model(model_name, model_path, dataset_dict):
    print(f"\n🔹 Evaluating {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForQuestionAnswering.from_pretrained(model_path)
    qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

    for ds_name, ds_path in dataset_dict.items():
        print(f"\n📘 Dataset: {ds_name}")
        dataset = json.load(open(ds_path, "r", encoding="utf-8"))
        flat_eval = flatten_squad(dataset)

        predictions, references = [], []

        for ex in tqdm(flat_eval):
            for qa in ex["qas"]:
                if qa.get("is_impossible", False):
                    predictions.append({
                        "id": qa["id"],
                        "prediction_text": "",
                        "no_answer_probability": 1.0
                    })
                    references.append({
                        "id": qa["id"],
                        "answers": {"text": [], "answer_start": []}
                    })
                else:
                    try:
                        pred = qa_pipeline(
                            question=qa["question"],
                            context=ex["context"],
                            handle_impossible_answer=True
                        )
                        predictions.append({
                            "id": qa["id"],
                            "prediction_text": pred["answer"],
                            "no_answer_probability": 1.0 - pred.get("score", 0.0)
                        })
                        references.append({
                            "id": qa["id"],
                            "answers": {
                                "text": [a["text"] for a in qa["answers"]],
                                "answer_start": [a["answer_start"] for a in qa["answers"]]
                            }
                        })
                    except Exception as e:
                        print(f"⚠️ Skipped QA {qa.get('id', 'unknown')}: {e}")

        results = metric.compute(predictions=predictions, references=references)
        print(f"\n✅ Results for {ds_name}:")
        print(results)  # 👈 useful to confirm the keys
        print(f"   • Exact Match (EM): {results.get('exact_match', results.get('exact', 0)):.2f}")
        print(f"   • F1 Score: {results.get('f1', 0):.2f}")

## Ejecutar para ambos modelos

In [25]:
# 🚀 Run for both models
for model_name, model_path in models.items():
    evaluate_model(model_name, model_path, datasets)


🔹 Evaluating BERT (Base)


Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



📘 Dataset: Colombia and Mexico Train


100%|██████████| 3235/3235 [2:19:24<00:00,  2.59s/it]  



✅ Results for Colombia and Mexico Train:
{'exact': 71.22425237801957, 'f1': 81.26662254248514, 'total': 14613, 'HasAns_exact': 58.60813072152771, 'HasAns_f1': 73.05336698625419, 'HasAns_total': 10159, 'NoAns_exact': 100.0, 'NoAns_f1': 100.0, 'NoAns_total': 4454, 'best_exact': 71.22425237801957, 'best_exact_thresh': 0.9643369317054749, 'best_f1': 81.26662254248333, 'best_f1_thresh': 0.9983832836151123}
   • Exact Match (EM): 71.22
   • F1 Score: 81.27

📘 Dataset: Colombia and Mexico Eval


...


✅ Results for Mexico Train:
{'exact': 88.33819241982508, 'f1': 92.34395934374338, 'total': 4116, 'HasAns_exact': 83.4653806407165, 'HasAns_f1': 89.14493167717815, 'HasAns_total': 2903, 'NoAns_exact': 100.0, 'NoAns_f1': 100.0, 'NoAns_total': 1213, 'best_exact': 88.33819241982508, 'best_exact_thresh': 0.9492854475975037, 'best_f1': 92.34395934374325, 'best_f1_thresh': 0.9998850226402283}
   • Exact Match (EM): 88.34
   • F1 Score: 92.34

📘 Dataset: Colombia Eval


100%|██████████| 2211/2211 [35:06<00:00,  1.05it/s]  



✅ Results for Colombia Eval:
{'exact': 78.0395643476328, 'f1': 84.12246541797725, 'total': 4499, 'HasAns_exact': 68.64487464297049, 'HasAns_f1': 77.33004503823568, 'HasAns_total': 3151, 'NoAns_exact': 100.0, 'NoAns_f1': 100.0, 'NoAns_total': 1348, 'best_exact': 78.0395643476328, 'best_exact_thresh': 0.9985507130622864, 'best_f1': 84.12246541797711, 'best_f1_thresh': 0.9998937249183655}
   • Exact Match (EM): 78.04
   • F1 Score: 84.12

📘 Dataset: Mexico Eval


100%|██████████| 880/880 [19:13<00:00,  1.31s/it]



✅ Results for Mexico Eval:
{'exact': 80.10204081632654, 'f1': 85.79334197348315, 'total': 1764, 'HasAns_exact': 71.27659574468085, 'HasAns_f1': 79.49218923177104, 'HasAns_total': 1222, 'NoAns_exact': 100.0, 'NoAns_f1': 100.0, 'NoAns_total': 542, 'best_exact': 80.10204081632654, 'best_exact_thresh': 0.9905273914337158, 'best_f1': 85.79334197348324, 'best_f1_thresh': 0.9998733401298523}
   • Exact Match (EM): 80.10
   • F1 Score: 85.79
